In [1]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
# from lxml import etree
import re
from tqdm import tqdm
import json
import random
from nltk.tokenize import word_tokenize

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
df = pd.read_csv('supp_names_ui_refer_ui.csv')

In [3]:
supp_list = df['supp_names'].tolist()

In [4]:
pattern_4 = '[A-Z][A-Z][A-Z][A-Z]'
acro_5 = []

for name in supp_list:
    if re.findall(pattern_4, name) and ',' not in name and '(' not in name:
        acro_5.append(name)
        
# acro_5

In [5]:
len(acro_5)

3433

In [7]:
pattern_3 = '[A-Z][A-Z][A-Z]'
acro_3 = []

for name in supp_list:
    if re.findall(pattern_3, name) and ',' not in name and '(' not in name:
        acro_3.append(name)
        
len(acro_3)        

13325

In [10]:
pattern_2 = '[A-Z][A-Z]'
acro_2 = []

for name in supp_list:
    if re.findall(pattern_2, name) and ',' not in name and '(' not in name:
        acro_2.append(name)
        
len(acro_2)  

22957

In [11]:
records = []
with open('BC7T2-NLMChem-corpus_v2.BioC.train.json', 'r') as f:
    records = [json.loads(line) for line in f]

In [12]:
# Tokenize the given replacement
def get_tokens(replacement):
    if '-' in replacement:
        t = replacement.split('-')
        tokens = ['-'] * (len(t) * 2 - 1)
        tokens[0::2] = t

        res = []
        for token in tokens:
            res += word_tokenize(token)
        tokens = res
    else:
        tokens = word_tokenize(replacement)
    
    for i, t in enumerate(tokens):
        if ',' in t:
            tokens = tokens[:i] + [c for c in t] + tokens[i + 1:]
            
    return tokens

In [13]:
# Tags for the replacement
def get_tags(replacement):
    ner_tags = []
    tokens = get_tokens(replacement)
    
    for i in range(len(tokens)):
        if i == 0:
            ner_tags.append('B-Chemical')
        else:
            ner_tags.append('I-Chemical')

    return ner_tags

In [14]:
# Spans for the replacement
def get_spans(replacement):
    spans = []
    span_0 = 0
    tokens = get_tokens(replacement)
    
    for t in tokens:
        if t == '-':
            spans += [[span_0 - 1, span_0]]
        elif t == ',':
            spans += [[span_0 - 1, span_0]]
        else:
            spans += [[span_0, span_0 + len(t)]]
            span_0 = span_0 + len(t) + 1

    return spans

In [27]:
# change acro for either 4 or 3 letter acronyms

def get_replacement(records):
    replacement = []
    num_chem = records['ner_tags'].count('B-Chemical')
    for i in range(num_chem):
        replacement.append(acro_3[random.randrange(0, n)])  
    return replacement

In [29]:
# a test case to check the three functions
n = len(acro_3)

for r in records[:1]:
    num_chem = r['ner_tags'].count('B-Chemical')
    replacement = []
    for i in range(num_chem):
        replacement.append(acro_3[random.randrange(0, n)])
    
replacement
get_tokens(replacement[0])
get_tags(replacement[0])
get_spans(replacement[0])

['ribosomal RNA apurinic site specific lyase', 'RPR 113228', 'WLBU2 peptide']

['ribosomal', 'RNA', 'apurinic', 'site', 'specific', 'lyase']

['B-Chemical',
 'I-Chemical',
 'I-Chemical',
 'I-Chemical',
 'I-Chemical',
 'I-Chemical']

[[0, 9], [10, 13], [14, 22], [23, 27], [28, 36], [37, 42]]

In [17]:
[get_spans(r) for r in replacement]

[[[0, 4], [5, 13]], [[0, 8]], [[0, 4], [4, 5], [5, 7], [8, 16]]]

In [18]:
def old_tag_index(r):
    old_tag_index = []
    for i, e in enumerate(r['ner_tags']):
        if e == 'B-Chemical':
            old_tag_index.append(i)
            
    return old_tag_index

In [19]:
def old_tag_index_dict(r):
    old_tag_index_dict = {}
    for i, e in enumerate(r['ner_tags']):
        if e == 'B-Chemical':
            old_tag_index_dict[i] = 1
            temp_i = i
        if e == 'I-Chemical':
            old_tag_index_dict[temp_i] += 1
            
    return old_tag_index_dict

In [20]:
# get the updated tags after inserting the replacement
r = records[0]

def updated_tags(r, replacement):
    old_tag_index_0 = old_tag_index(r)
    old_tag_index_dict_0 = old_tag_index_dict(r)
#     for i, e in enumerate(r['ner_tags']):
#         if e == 'B-Chemical':
#             old_tag_index.append(i)
#             old_tag_index_dict[i] = 1
#             temp_i = i
#         if e == 'I-Chemical':
#             old_tag_index_dict[temp_i] += 1
        
    new_tags = []
    for i in range(len(replacement)):
        new_tags.append(get_tags(replacement[i]))

    start = 0
    updated_tags = []
    for i, re in enumerate(replacement):
        temp_tags = r['ner_tags'][start : old_tag_index_0[i]]
        temp_tags.extend(new_tags[i])
        updated_tags.extend(temp_tags)
        start = old_tag_index_0[i] + 1

        # when there are I-Chemical，start is moved to the next ‘O’
        # if there is no 'O' after all the I-Chemical
        while start < len(r['ner_tags']) and r['ner_tags'][start] == 'I-Chemical':
            start += 1

    # if there are 'O' left, add them on
    if start == len(r['ner_tags']):
        return updated_tags
    if r['ner_tags'][start]:
        updated_tags.extend(r['ner_tags'][start:])

    return updated_tags

In [21]:
replacement_760 = get_replacement(records[760])
replacement_760
# updated_tags(records[760], replacement_760)

['PSORI-CM01', 'Tris-borate-EDTA buffer']

In [22]:
updated_tags(records[760], replacement_760)

['O',
 'O',
 'O',
 'O',
 'O',
 'B-Chemical',
 'I-Chemical',
 'I-Chemical',
 'O',
 'B-Chemical',
 'I-Chemical',
 'I-Chemical',
 'I-Chemical',
 'I-Chemical',
 'I-Chemical']

In [18]:
# # test case for the updated tags
# r['ner_tags']
# updated_tags(r, replacement)
# len(updated_tags(r, replacement))

In [23]:
# update the tokens
def updated_tokens(r, replacement):
    new_tokens = []

    for i in range(len(replacement)):
        new_tokens.append(get_tokens(replacement[i]))

    start = 0
    updated_tokens = []

    for i, re in enumerate(replacement):
        end = old_tag_index(r)[i]
        temp_tokens = r['tokens'][start: end]
        temp_tokens.extend(new_tokens[i])

        updated_tokens.extend(temp_tokens)
        start = end + 1
        
        while start < len(r['ner_tags']) and r['ner_tags'][start] == 'I-Chemical':
            start += 1

    # If there are ‘O', add the corresponding tokens
    if start == len(r['ner_tags']):
        return updated_tokens
    if r['tokens'][start]:
        updated_tokens.extend(r['tokens'][start:])

    return updated_tokens

In [24]:
replacement_760 = get_replacement(records[760])
replacement_760

['CHIR98023', 'SLUPP-225']

In [25]:
updated_tokens(records[760], replacement_760)

['Identification',
 'of',
 'the',
 'proteins',
 'bearing',
 'CHIR98023',
 'branched',
 'SLUPP',
 '-',
 '225']

In [22]:
# # a test case for updated_tokens
# updated_tokens(r, replacement)
# len(updated_tokens(r, replacement))

#### Updating the spans (tricky)

In [23]:
old_tag_index(r)
old_tag_index_dict(r)

[6, 8, 10]

{6: 1, 8: 1, 10: 1}

In [24]:
# r

In [25]:
# update the spans
replacement

def getSpans(r, replacement):
    r_spans = r['spans']
    r_tags = r['ner_tags']
    old_tag_index_0 = old_tag_index(r)
    old_tag_index_dict_0 = old_tag_index_dict(r)
    
    new_spans = []
    for i in range(len(replacement)):
        new_spans.append(get_spans(replacement[i]))

    start = 0
    for i, re in enumerate(replacement):
        updated_spans = []
        len_replacement = len(new_spans[i])
        end = old_tag_index_0[i]
        
        if i == 0:
            temp_spans = r_spans[:end]
            for j in range(len_replacement):
                ss = np.add([r_spans[end][0], r_spans[end][0]], new_spans[i][j])
                updated_spans.append(ss.tolist())
        else:
            # now, use the old_tag_index_dict to find the next chemical span and update with the value in the temp_spans
            start = old_tag_index_0[i]
            space = r_spans[start][0] - r_spans[start - 1][1]

            # update the spans with the new_spans, space and temp_spans[-1][1] = 3560
            for j in range(len_replacement):
                ss = np.add([temp_spans[-1][1] + space, temp_spans[-1][1] + space], new_spans[i][j])
                updated_spans.append(ss.tolist())

        # add the new spans (of tokens) to temp_spans
        temp_spans.extend(updated_spans)

        # extend the spans with 'O' entities if there are 'O''s behind，old spans: [3578, 3579] --> updated_o_spans
        # use old_tag_index_dict to get rid of the old entity, it tells you the length of the old chemical entity
        o_start = old_tag_index_0[i] + old_tag_index_dict_0[old_tag_index_0[i]]
        if i + 1 < len(old_tag_index_0):
            o_end = old_tag_index_0[i + 1]
        else:
            o_end = len(r_spans)

        o_span = r_spans[o_start: o_end]
        # update the o_span and extend the temp_spans
#         print('o_start: ', o_start)
        if o_start == len(r['ner_tags']):
            return temp_spans
        else:
            space = r_spans[o_start][0] - r_spans[o_start - 1][1]
            updated_o_spans = []
            diff = temp_spans[-1][1] - r_spans[o_start - 1][1]

            for k in range(o_start, o_end):
                ss = np.add(r_spans[k], [diff, diff])
                updated_o_spans.append(list(ss.tolist()))

            if updated_o_spans:
                temp_spans.extend(updated_o_spans)

#     print('========= result ==========')
    return temp_spans

['HIPRA SARS-CoV-2 vaccine', 'VUFB 17951', 'YHHU0895']

In [28]:
# getSpans(records[760], replacement_760)

In [29]:
# getSpans(r, replacement)
# len(getSpans(r, replacement))

In [30]:
# form a dict of r
# r
# new_r = dict()
# new_r['ner_tags'] = updated_tags(r, replacement)
# new_r['tokens'] = updated_tokens(r, replacement)
# new_r['spans'] = getSpans(r, replacement)
# new_r

In [30]:
# new_replacement = ['Carbaryl', 'Naphthalene', 'Chlorpyrifos']

In [31]:
# getSpans(new_r, new_replacement)

In [32]:
# getSpans(new_r, new_replacement) == r['spans']

In [33]:
def new_records(r, replacement):
    new_r = dict()
    new_r['ner_tags'] = updated_tags(r, replacement)
    new_r['tokens'] = updated_tokens(r, replacement)
    new_r['spans'] = getSpans(r, replacement)
    return new_r

In [34]:
# extract the original chemical entities using ner_tags, tokens, and spans
# get the index of 'B-Chemical' and following 'I-Chemical' and their spans, put them together as the replacement

def original_chemical(r):
    tokens = []
    old_index = old_tag_index(r)
    old_index_dict = old_tag_index_dict(r)
    
    for i in old_index:
        temp = []
        temp.append(r['tokens'][i])
        j = i + 1
        n = 1
        while n < old_index_dict[i]:
            space = r['spans'][j][0] - r['spans'][j - 1][1]
            next_tag = r['ner_tags'][j]
            if next_tag == 'I-Chemical':
                if space == 1:
                    temp[-1] = temp[-1] + ' ' + r['tokens'][j]
                if space == 0:
                    temp[-1] = temp[-1] + r['tokens'][j]
                j += 1
                n += 1
        tokens.extend(temp)
        
    return tokens

In [35]:
r = records[0]

In [36]:
original_chemical(r)

['Carbaryl', 'Naphthalene', 'Chlorpyrifos']

In [37]:
# r_1 = {'ner_tags': ['O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'I-Chemical', 'I-Chemical', 'O', 'B-Chemical', 'I-Chemical', 'O', 'B-Chemical', 'I-Chemical', 'I-Chemical', 'I-Chemical', 'I-Chemical', 'I-Chemical', 'I-Chemical', 'O', 'O', 'O', 'O'], 'tokens': ['The', 'Relationship', 'of', 'Urinary', 'Metabolites', 'of', 'Gd', '-', 'TEMDO', '/', 'TOPRO', '5', 'and', 'DNA', 'excision', 'repair', 'protein', 'ERCC', '-', '5', 'with', 'Human', 'Semen', 'Quality'], 'spans': [[0, 3], [4, 16], [17, 19], [20, 27], [28, 39], [40, 42], [43, 45], [45, 46], [46, 51], [51, 52], [52, 57], [58, 59], [60, 63], [64, 67], [68, 76], [77, 83], [84, 91], [92, 96], [96, 97], [97, 98], [99, 103], [104, 109], [110, 115], [116, 123]]}

In [38]:
# original_chemical(r_1)

### Round tripping tests on spans

In [39]:
# replacement_list = get_replacement(records)

In [39]:
count = 0
total = 0
error_case = []
for i, r in enumerate(records[7061:]):
#     print(r)
    if 'B-Chemical' in r['ner_tags']:
        total += 1
        replacement = get_replacement(r)
#         print(replacement)
        new_r = new_records(r, replacement)
#         print(new_r)
        original_tokens = original_chemical(r)
#         print('original tokens', original_tokens)
#         getSpans(new_r, original_tokens)
        
        if getSpans(new_r, original_tokens) == r['spans']:
            count += 1
        else:
            error_case.append(i)
            continue
            
print('Matched', count)
print('Not matched: ', total - count)
print('Number of records that contain chemical entities', total)

Matched 5536
Not matched:  677
Number of records that contain chemical entities 6213


In [105]:
len(records)

18910

In [42]:
def roundtripping(records):
    count = 0
    total = 0
    error_case = []
    correct_case = []
    for i, r in enumerate(records):
        if 'B-Chemical' in r['ner_tags']:
            total += 1
            replacement = get_replacement(r)
            new_r = new_records(r, replacement)
            original_tokens = original_chemical(r)
#             print('replacement:', replacement)
#             print('new_r: ', new_r)
#             print('original_tokens: ', original_tokens)
#             print('len original spans: ', len(r['spans']))
#             print('reversed: ', getSpans(new_r, original_tokens))
#             print('len reversed: ', len(getSpans(new_r, original_tokens)))
            
            if getSpans(new_r, original_tokens) == r['spans']:
                count += 1
                correct_case.append(i)
            else:
                error_case.append(i)
                continue
    
    print('Matched', count)
    print('Not matched: ', total - count)
    print('Number of records that contain chemical entities', total)
    
    return correct_case
    

In [43]:
correct_0 = roundtripping(records[:7060])

Matched 2924
Not matched:  260
Number of records that contain chemical entities 3184


In [44]:
correct_1 = roundtripping(records[7061:])

Matched 5536
Not matched:  677
Number of records that contain chemical entities 6213


In [ ]:
correct_1

In [ ]:
records[7061]

In [89]:
correct_0[-1]
correct_1[0]

7057

0

In [54]:
correct_2 = [i + 7061 for i in correct_1]

### Get the correct cases ready for training

In [ ]:
records[-1]

In [40]:
# use the index to get the original records and convert to new ones

# ner_tags = []
# tokens = []
# spans = []
# add_records = []

def new_add_records(correct_i, id_0, doc_id):
    add_records = []
    for i in correct_i:
        record = records[i]
        temp_d = {}
        temp_d['id'] = id_0
        temp_d['document_id'] = doc_id
        replacement = get_replacement(record)
        ner_tags = updated_tags(record, replacement)
        tokens = updated_tokens(record, replacement)
        temp_d['ner_tags'] = ner_tags
        temp_d['tokens'] = tokens
        temp_d['spans'] = getSpans(record, replacement)
        add_records.append(temp_d)

        id_0 += 1
        doc_id += 1
    return add_records

In [173]:
id_0 = 18909
doc_id = 6032168

add_train_0 = new_add_records(correct_0, id_0, doc_id)

In [174]:
add_train_0[-1]

{'id': 21832,
 'document_id': 6035091,
 'ner_tags': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-Chemical',
  'I-Chemical',
  'I-Chemical',
  'I-Chemical',
  'I-Chemical',
  'I-Chemical',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 'tokens': ['Data',
  'from',
  'this',
  'study',
  'suggest',
  'that',
  'application',
  'of',
  'FL',
  '-',
  'BOND',
  'S',
  '-',
  '1',
  'prior',
  'to',
  'or',
  'in',
  'combination',
  'with',
  'disinfectants',
  'or',
  'antibiotics',
  'may',
  'allow',
  'for',
  'novel',
  'and',
  'improved',
  'strategies',
  'to',
  'control',
  'biofilms',
  'in',
  'industrial',
  'as',
  'well',
  'as',
  'clinical',
  'settings',
  ',',
  'with',
  'clear',
  'benefits',
  'such',
  'as',
  'reduced',


In [175]:
len(add_train_0)

2924

In [176]:
id_1 = 21833
doc_id_1 = 6035092
add_train_1 = new_add_records(correct_2, id_1, doc_id_1)

In [177]:
len(add_train_1)

5536

In [178]:
# get them into json format
add_train_1105 = add_train_0 + add_train_1
len(add_train_1105)

8460

In [ ]:
with open('add_train_1105.json', 'w') as fout:
    for d in add_train_1105:
        fout.write(json.dumps(d))
        fout.write('\n')

In [59]:
# get 4230 records - 1108
id_0 = 18909
doc_id = 6032168
add_train_2 = new_add_records(correct_0, id_0, doc_id)

id_1 = 21833
doc_id_1 = 6035092
add_train_3 = new_add_records(correct_2[:1306], id_1, doc_id_1)

add_train_1108 = add_train_2 + add_train_3
len(add_train_1108)

4230

In [ ]:
with open('add_train_1108.json', 'w') as fout:
    for d in add_train_1108:
        fout.write(json.dumps(d))
        fout.write('\n')

In [62]:
# get 16920 records - 1108
id_0 = 18909
doc_id = 6032168
add_train_4 = new_add_records(correct_0, id_0, doc_id)

id_1 = 21833
doc_id_1 = 6035092
add_train_5 = new_add_records(correct_2, id_1, doc_id_1)

id_2 = 27369
doc_id_2 = 6040628
add_train_6 = new_add_records(correct_0, id_2, doc_id_2)

id_3 = 30293
doc_id_3 = 6043552
add_train_7 = new_add_records(correct_2, id_3, doc_id_3)


add_train_1108_d = add_train_4 + add_train_5 + add_train_6 + add_train_7
len(add_train_1108_d)

16920

In [ ]:
with open('add_train_1108_d.json', 'w') as fout:
    for d in add_train_1108_d:
        fout.write(json.dumps(d))
        fout.write('\n')

In [68]:
def new_add_records_doc(correct_i, id_0):
    add_records = []
    for i in correct_i:
        record = records[i]
        temp_d = {}
        temp_d['id'] = id_0
        temp_d['document_id'] = str(int(record['document_id']) - 1253656 + 6032168)
        replacement = get_replacement(record)
        ner_tags = updated_tags(record, replacement)
        tokens = updated_tokens(record, replacement)
        temp_d['ner_tags'] = ner_tags
        temp_d['tokens'] = tokens
        temp_d['spans'] = getSpans(record, replacement)
        add_records.append(temp_d)

        id_0 += 1
    return add_records

In [74]:
### fix the document_id
# get 4230 records - 1108
id_4 = 18909
add_train_docid_0 = new_add_records_doc(correct_0, id_4)

id_5 = 21833
add_train_docid_1 = new_add_records_doc(correct_2[:1306], id_5)

add_train_docid = add_train_docid + add_train_docid_1

In [75]:
len(add_train_docid)

4230

In [ ]:
with open('add_train_1109.json', 'w') as fout:
    for d in add_train_docid:
        fout.write(json.dumps(d))
        fout.write('\n')

### Produce addition training data with 3 letter acronyms

In [26]:
acro_3

['ICM0301B',
 'JBIR-130',
 'JBIR-131',
 'JBIR-132',
 'CMESA-sepharose',
 'MON 0856',
 'MDL 73975',
 'beta-lactamase IRT-3',
 'BAL 19403',
 'IMAP protocol',
 'EXP 631',
 'HEOM',
 '6-amino-NAD',
 'FITC-tyramide',
 'DONS',
 'GEA 968',
 'NTE 122',
 'alpha-conotoxin MII',
 "poly 2'-azido-2'-deoxyAMP",
 'Fuji VII',
 'EUK-172',
 'TAC 101',
 'Amberlite XAD-2 resin',
 'herpes simplex virus type 2 protein ICSP 11-12',
 'cinnamoyl-LIGRLO-amide',
 'UDPglucose-glycoprotein glucose-1-phosphotransferase',
 'UDPgalactose - UDP-N-acetylglucosamine galactosephosphotransferase',
 'ICR 340',
 '2-chloro-ATP',
 'FCE 27677',
 'cyclo-CTP',
 'heliogen blue SBL',
 'GKPV peptide',
 'MTP 1403',
 'glycoprotein BCA 225',
 'dTDP-L-dihydrostreptose-streptidine-6-phosphate dihydrostreptosyltransferase',
 'BODIPY-PC',
 'PEN protocol',
 'BMP-PEI-pSLC-E7-Fc vaccine',
 'APO 10',
 'ITA 529',
 'CGP 62349',
 'GAHKNYLRFamide',
 'CGP 64213',
 'ONCOFID-S',
 'NIR-mbc94',
 'AZD3409',
 'trisilanolisobutyl-POSS',
 'ORF 8511',
 'RPR